<a href="https://colab.research.google.com/github/deepsree24/Fake-User-Profile-Detection/blob/master/fakeprofiledetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas_profiling as pp
import keras as k


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
df_users = pd.read_csv("/content/users.csv")
df_fusers = pd.read_csv("/content/fusers.csv")

In [ ]:
df_fusers.shape

(1337, 34)

In [ ]:
df_users.shape

(1481, 34)

In [ ]:
#for df_fusers
isFake = np.zeros(1337)

#for df_users
isNotFake = np.ones(1481)

In [ ]:
df_fusers["isFake"] = isFake
df_users["isFake"] = isNotFake

In [ ]:
df_allUsers = pd.concat([df_fusers, df_users], ignore_index=True)
df_allUsers.columns = df_allUsers.columns.str.strip()

In [ ]:
df_allUsers = df_allUsers.sample(frac=1).reset_index(drop=True)

In [ ]:
df_allUsers.describe()

,id,statuses_count,followers_count,friends_count,favourites_count,listed_count,default_profile,default_profile_image,geo_enabled,profile_use_background_image,profile_background_tile,utc_offset,protected,verified,isFake
count,2.818000e+03,2818.000000,2818.000000,2818.000000,2818.000000,2818.000000,1728.0,8.0,721.0,2760.0,489.0,1069.000000,0.0,0.0,2818.000000
mean,5.374889e+08,1672.198368,371.105039,395.363023,234.541164,2.818666,1.0,1.0,1.0,1.0,1.0,1478.391020,NaN,NaN,0.525550
std,2.977005e+08,4884.669157,8022.631339,465.694322,1445.847248,23.480430,0.0,0.0,0.0,0.0,0.0,8108.211889,NaN,NaN,0.499435
min,3.610511e+06,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0,1.0,1.0,1.0,-39600.000000,NaN,NaN,0.000000
25%,3.620867e+08,35.000000,17.000000,168.000000,0.000000,0.000000,1.0,1.0,1.0,1.0,1.0,3600.000000,NaN,NaN,0.000000
50%,6.162253e+08,77.000000,26.000000,306.000000,0.000000,0.000000,1.0,1.0,1.0,1.0,1.0,3600.000000,NaN,NaN,1.000000
75%,6.177673e+08,1087.750000,111.000000,519.000000,37.000000,1.000000,1.0,1.0,1.0,1.0,1.0,3600.000000,NaN,NaN,1.000000
max,1.391998e+09,79876.000000,408372.000000,12773.000000,44349.000000,744.000000,1.0,1.0,1.0,1.0,1.0,36000.000000,NaN,NaN,1.000000


In [ ]:
df_allUsers.head()


,id,name,screen_name,statuses_count,followers_count,friends_count,favourites_count,listed_count,created_at,url,lang,time_zone,location,default_profile,default_profile_image,geo_enabled,profile_image_url,profile_banner_url,profile_use_background_image,profile_background_image_url_https,profile_text_color,profile_image_url_https,profile_sidebar_border_color,profile_background_tile,profile_sidebar_fill_color,profile_background_image_url,profile_background_color,profile_link_color,utc_offset,protected,verified,description,updated,dataset,isFake
0,87654715,giampaolo galenda,p1qu3,299,29,28,3,0,Thu Nov 05 09:59:29 +0000 2009,http://totallyunnecessaryproductions.it,en,Rome,creative director,NaN,NaN,1.0,http://a0.twimg.com/profile_images/1901157677/...,https://si0.twimg.com/profile_banners/87654715...,1.0,https://si0.twimg.com/images/themes/theme5/bg.gif,3E4415,https://si0.twimg.com/profile_images/190115767...,829D5E,NaN,99CC33,http://a0.twimg.com/images/themes/theme5/bg.gif,352726,D02B55,3600.0,NaN,NaN,Creativity is allowing yourself to make mistak...,2015-02-14 10:54:49,E13,1.0
1,428904234,Francesco Genova,gengio5,1479,13,25,47,0,Mon Dec 05 09:44:07 +0000 2011,NaN,it,NaN,NaN,NaN,NaN,NaN,http://a0.twimg.com/profile_images/2655673055/...,https://si0.twimg.com/profile_banners/42890423...,1.0,https://si0.twimg.com/profile_background_image...,333333,https://si0.twimg.com/profile_images/265567305...,C0DEED,NaN,DDEEF6,http://a0.twimg.com/profile_background_images/...,CC0033,CC3300,NaN,NaN,NaN,"Magno nutella e amatriciana,tifo Roma e canto ...",2015-02-14 10:54:49,E13,1.0
2,619735156,Teri Albert,terialbertyykp,29,14,284,0,0,Wed Jun 27 06:19:46 +0000 2012,NaN,en,NaN,Houston but should be NEWYORK,1.0,NaN,NaN,http://a0.twimg.com/profile_images/3303656739/...,NaN,1.0,https://si0.twimg.com/images/themes/theme1/bg.png,333333,https://si0.twimg.com/profile_images/330365673...,C0DEED,NaN,DDEEF6,http://a0.twimg.com/images/themes/theme1/bg.png,C0DEED,0084B4,NaN,NaN,NaN,#team MODEL #teamBBM #teamjesus I AM THE NEW...,2015-02-14 10:40:01,INT,0.0
3,464790896,Flavia Casanova,Flavia1105,1402,150,179,244,1,Sun Jan 15 16:44:55 +0000 2012,NaN,it,London,NaN,NaN,NaN,NaN,http://a0.twimg.com/profile_images/1761244994/...,NaN,1.0,https://si0.twimg.com/images/themes/theme4/bg.gif,3C3940,https://si0.twimg.com/profile_images/176124499...,5ED4DC,NaN,95E8EC,http://a0.twimg.com/images/themes/theme4/bg.gif,0099B9,0099B9,0.0,NaN,NaN,Sono nata e vivo in provincia di Genova;da pic...,2015-02-14 10:54:49,E13,1.0
4,383292062,ilPux,MarcoPusceddu1,222,125,300,20,0,Sat Oct 01 16:25:48 +0000 2011,NaN,it,NaN,NaN,NaN,NaN,NaN,http://a0.twimg.com/profile_images/2674480298/...,NaN,1.0,https://si0.twimg.com/images/themes/theme15/bg...,333333,https://si0.twimg.com/profile_images/267448029...,A8C7F7,NaN,C0DFEC,http://a0.twimg.com/images/themes/theme15/bg.png,022330,0084B4,NaN,NaN,NaN,"Studente, pallavolista, Scout, divoratore di l...",2015-02-14 10:54:49,E13,1.0


In [ ]:
Y = df_allUsers.isFake

In [ ]:
df_allUsers.drop(["isFake"],axis=1, inplace=True)
X = df_allUsers

In [ ]:
pp.ProfileReport(X)

In [ ]:
Y.reset_index(drop=True, inplace=True)

In [ ]:
print(Y.shape)

(2818,)


In [ ]:
X.head()

,id,name,screen_name,statuses_count,followers_count,friends_count,favourites_count,listed_count,created_at,url,lang,time_zone,location,default_profile,default_profile_image,geo_enabled,profile_image_url,profile_banner_url,profile_use_background_image,profile_background_image_url_https,profile_text_color,profile_image_url_https,profile_sidebar_border_color,profile_background_tile,profile_sidebar_fill_color,profile_background_image_url,profile_background_color,profile_link_color,utc_offset,protected,verified,description,updated,dataset
0,87654715,giampaolo galenda,p1qu3,299,29,28,3,0,Thu Nov 05 09:59:29 +0000 2009,http://totallyunnecessaryproductions.it,en,Rome,creative director,NaN,NaN,1.0,http://a0.twimg.com/profile_images/1901157677/...,https://si0.twimg.com/profile_banners/87654715...,1.0,https://si0.twimg.com/images/themes/theme5/bg.gif,3E4415,https://si0.twimg.com/profile_images/190115767...,829D5E,NaN,99CC33,http://a0.twimg.com/images/themes/theme5/bg.gif,352726,D02B55,3600.0,NaN,NaN,Creativity is allowing yourself to make mistak...,2015-02-14 10:54:49,E13
1,428904234,Francesco Genova,gengio5,1479,13,25,47,0,Mon Dec 05 09:44:07 +0000 2011,NaN,it,NaN,NaN,NaN,NaN,NaN,http://a0.twimg.com/profile_images/2655673055/...,https://si0.twimg.com/profile_banners/42890423...,1.0,https://si0.twimg.com/profile_background_image...,333333,https://si0.twimg.com/profile_images/265567305...,C0DEED,NaN,DDEEF6,http://a0.twimg.com/profile_background_images/...,CC0033,CC3300,NaN,NaN,NaN,"Magno nutella e amatriciana,tifo Roma e canto ...",2015-02-14 10:54:49,E13
2,619735156,Teri Albert,terialbertyykp,29,14,284,0,0,Wed Jun 27 06:19:46 +0000 2012,NaN,en,NaN,Houston but should be NEWYORK,1.0,NaN,NaN,http://a0.twimg.com/profile_images/3303656739/...,NaN,1.0,https://si0.twimg.com/images/themes/theme1/bg.png,333333,https://si0.twimg.com/profile_images/330365673...,C0DEED,NaN,DDEEF6,http://a0.twimg.com/images/themes/theme1/bg.png,C0DEED,0084B4,NaN,NaN,NaN,#team MODEL #teamBBM #teamjesus I AM THE NEW...,2015-02-14 10:40:01,INT
3,464790896,Flavia Casanova,Flavia1105,1402,150,179,244,1,Sun Jan 15 16:44:55 +0000 2012,NaN,it,London,NaN,NaN,NaN,NaN,http://a0.twimg.com/profile_images/1761244994/...,NaN,1.0,https://si0.twimg.com/images/themes/theme4/bg.gif,3C3940,https://si0.twimg.com/profile_images/176124499...,5ED4DC,NaN,95E8EC,http://a0.twimg.com/images/themes/theme4/bg.gif,0099B9,0099B9,0.0,NaN,NaN,Sono nata e vivo in provincia di Genova;da pic...,2015-02-14 10:54:49,E13
4,383292062,ilPux,MarcoPusceddu1,222,125,300,20,0,Sat Oct 01 16:25:48 +0000 2011,NaN,it,NaN,NaN,NaN,NaN,NaN,http://a0.twimg.com/profile_images/2674480298/...,NaN,1.0,https://si0.twimg.com/images/themes/theme15/bg...,333333,https://si0.twimg.com/profile_images/267448029...,A8C7F7,NaN,C0DFEC,http://a0.twimg.com/images/themes/theme15/bg.png,022330,0084B4,NaN,NaN,NaN,"Studente, pallavolista, Scout, divoratore di l...",2015-02-14 10:54:49,E13


In [ ]:
lang_list = list(enumerate(np.unique(X["lang"])))
lang_dict = {name : i for i, name in lang_list}
X.loc[:, "lang_num"] = X["lang"].map(lambda x: lang_dict[x]).astype(int)

X.drop(["name"], axis=1, inplace=True)

In [ ]:
X = X[[
    "statuses_count",
    "followers_count",
    "friends_count",
    "favourites_count",
    "lang_num",
    "listed_count",
    "geo_enabled",
    "utc_offset",
    "profile_use_background_image"
                        ]]

In [ ]:
pp.ProfileReport(X)

In [ ]:
X = X.replace(np.nan, 0)

In [ ]:
pp.ProfileReport(X)

In [ ]:
train_X, val_X, train_y, val_y = train_test_split(X, Y, train_size=0.8, test_size=0.2, random_state=0)

In [ ]:
print(train_X.shape)
print(val_X.shape)
print(train_y.shape)
print(val_y.shape)

(2254, 9)
(564, 9)
(2254,)
(564,)


In [ ]:
from numpy.random import seed
seed(1)

In [ ]:
import keras
from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras import optimizers

In [ ]:
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=9))
model.add(Dense(64, input_dim=32,
                kernel_regularizer=regularizers.l2(0.0076),
                activity_regularizer=regularizers.l1(0.0023), activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_X, train_y, epochs=8)

# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

Epoch 1/8
2254/2254 [==============================] - 0s 81us/step - loss: 259.7344 - accuracy: 0.8017
Epoch 2/8
2254/2254 [==============================] - 0s 39us/step - loss: 27.7914 - accuracy: 0.7533
Epoch 3/8
2254/2254 [==============================] - 0s 39us/step - loss: 9.7162 - accuracy: 0.9445
Epoch 4/8
2254/2254 [==============================] - 0s 37us/step - loss: 3.5624 - accuracy: 0.7746
Epoch 5/8
2254/2254 [==============================] - 0s 37us/step - loss: 2.0905 - accuracy: 0.9814
Epoch 6/8
2254/2254 [==============================] - 0s 36us/step - loss: 1.3352 - accuracy: 0.9760
Epoch 7/8
2254/2254 [==============================] - 0s 38us/step - loss: 0.7147 - accuracy: 0.9823
Epoch 8/8
2254/2254 [==============================] - 0s 37us/step - loss: 0.7239 - accuracy: 0.9521


In [ ]:
score, acc = model.evaluate(val_X, val_y)
print(acc*100)

564/564 [==============================] - 0s 57us/step
98.93617033958435
